In [1]:
# Taken from GPYTorch Examples

In [6]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

In [7]:
import urllib.request
import os
from scipy.io import loadmat
from math import floor

data = torch.Tensor(loadmat('./elevators.mat/elevators.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]


train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

#if torch.cuda.is_available():
#    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

In [8]:
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal

class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.base_covar_module = ScaleKernel(RBFKernel())
        self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=train_x[:500, :], likelihood=likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [9]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

#if torch.cuda.is_available():
#    model = model.cuda()
#    likelihood = likelihood.cuda()

In [19]:
for param_name, param in model.named_parameters():
    print(param_name)
    print(param.size())

likelihood.noise_covar.raw_noise
torch.Size([1])
mean_module.raw_constant
torch.Size([])
base_covar_module.raw_outputscale
torch.Size([])
base_covar_module.base_kernel.raw_lengthscale
torch.Size([1, 1])
covar_module.inducing_points
torch.Size([500, 18])


In [16]:

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

def train():
    for i in range(50):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, 50, loss.item()))
        optimizer.step()
        torch.cuda.empty_cache()

# See dkl_mnist.ipynb for explanation of this flag
%time train()

Iter 1/50 - Loss: 0.799
Iter 2/50 - Loss: 0.793
Iter 3/50 - Loss: 0.787
Iter 4/50 - Loss: 0.782
Iter 5/50 - Loss: 0.777
Iter 6/50 - Loss: 0.772
Iter 7/50 - Loss: 0.767
Iter 8/50 - Loss: 0.763
Iter 9/50 - Loss: 0.758
Iter 10/50 - Loss: 0.753
Iter 11/50 - Loss: 0.749
Iter 12/50 - Loss: 0.744
Iter 13/50 - Loss: 0.740
Iter 14/50 - Loss: 0.736
Iter 15/50 - Loss: 0.731
Iter 16/50 - Loss: 0.727
Iter 17/50 - Loss: 0.722
Iter 18/50 - Loss: 0.718
Iter 19/50 - Loss: 0.714
Iter 20/50 - Loss: 0.709
Iter 21/50 - Loss: 0.705
Iter 22/50 - Loss: 0.701
Iter 23/50 - Loss: 0.697
Iter 24/50 - Loss: 0.692
Iter 25/50 - Loss: 0.688
Iter 26/50 - Loss: 0.684
Iter 27/50 - Loss: 0.680
Iter 28/50 - Loss: 0.676
Iter 29/50 - Loss: 0.671
Iter 30/50 - Loss: 0.667
Iter 31/50 - Loss: 0.663
Iter 32/50 - Loss: 0.659
Iter 33/50 - Loss: 0.654
Iter 34/50 - Loss: 0.650
Iter 35/50 - Loss: 0.646
Iter 36/50 - Loss: 0.642
Iter 37/50 - Loss: 0.638
Iter 38/50 - Loss: 0.633
Iter 39/50 - Loss: 0.629
Iter 40/50 - Loss: 0.625
Iter 41/5

In [17]:
model.eval()
likelihood.eval()
with gpytorch.settings.max_preconditioner_size(10), torch.no_grad():
    preds = model(test_x)

In [18]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))


Test MAE: 0.0731787383556366
